In [86]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
from datetime import timedelta
import datetime as dt

## 1. Navigate to https://www.billboard.com/charts/hot-100/. 
Using BeautifulSoup, extract out the This Week, artist, song, Last Week, Peak Position, and Weeks on Chart values into a pandas DataFrame. 
Hint: The HTML for the number one ranked song is slightly different from that of the rest of the songs.

In [6]:
response = requests.get('https://www.billboard.com/charts/hot-100/')

In [7]:
soup = BS(response.text)

In [10]:
hot_100 = soup.findAll('div', attrs = {'class': 'o-chart-results-list-row-container'})

In [52]:
hot100 = []
i = 0
for x in hot_100:
    title = hot_100[i].find('h3').text.strip()
    artist = hot_100[i].find('span', attrs = {'class': 'a-no-trucate'}).text.strip()
    this_week = hot_100[i].find('span').text.strip()
    last_week = hot_100[i].findAll('span', attrs = {'class': 'u-font-family-secondary@mobile-max'})[0].text.strip()
    peak = hot_100[i].findAll('span', attrs = {'class': 'u-font-family-secondary@mobile-max'})[1].text.strip()
    weeks = hot_100[i].findAll('span', attrs = {'class': 'u-font-family-secondary@mobile-max'})[2].text.strip()

    hot100.append({'Song': title, 'Artist': artist, 'This Week': this_week, 'Last Week': last_week, 'Peak Position': peak, 'Weeks on Chart': weeks})

    i+=1

hot100 = pd.DataFrame(hot100)

In [53]:
hot100

,Song,Artist,This Week,Last Week,Peak Position,Weeks on Chart
0,The Fate Of Ophelia,Taylor Swift,1,1,1,7
1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",2,2,1,22
2,Ordinary,Alex Warren,3,3,1,41
3,Man I Need,Olivia Dean,4,4,4,13
4,Opalite,Taylor Swift,5,5,2,7
...,...,...,...,...,...,...
95,Baller,"Summer Walker, Glorilla, Sexyy Red & Monaleo",96,-,96,1
96,Let Alone The One You Love,Olivia Dean,97,-,97,1
97,Losing Streak,Blake Roman,98,-,98,1
98,Favorite Country Song,HARDY,99,90,90,4


## 2. After getting the code working for the current chart, navigate to last week's chart. Notice how the url for the page changes. 
Write a function which will, given a date, return a pandas DataFrame containing the Billboard chart data for that date.

In [75]:
def billboard(date):
    ## must be passed a Saturday, in YYYY-MM-DD format as a string
    ## (this is a webscraping project not a datetime formatting project, so I'm not messing with making it more flexible right now)
    
    soup = BS(requests.get('https://www.billboard.com/charts/hot-100/'+str(date)).text)
    hot_100 = soup.findAll('div', attrs = {'class': 'o-chart-results-list-row-container'})

    hot100 = []
    i = 0
    for x in hot_100:
        title = hot_100[i].find('h3').text.strip()
        artist = hot_100[i].find('span', attrs = {'class': 'a-no-trucate'}).text.strip()
        this_week = hot_100[i].find('span').text.strip()
        last_week = hot_100[i].findAll('span', attrs = {'class': 'u-font-family-secondary@mobile-max'})[0].text.strip()
        peak = hot_100[i].findAll('span', attrs = {'class': 'u-font-family-secondary@mobile-max'})[1].text.strip()
        weeks = hot_100[i].findAll('span', attrs = {'class': 'u-font-family-secondary@mobile-max'})[2].text.strip()
    
        hot100.append({'Week': date, 'Song': title, 'Artist': artist, 'This Week': this_week, 'Last Week': last_week, 'Peak Position': peak, 'Weeks on Chart': weeks})

        i+=1

    hot100 = pd.DataFrame(hot100)
    return hot100

In [76]:
billboard('2025-10-08')

,Week,Song,Artist,This Week,Last Week,Peak Position,Weeks on Chart
0,2025-10-08,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",1,1,1,15
1,2025-10-08,Ordinary,Alex Warren,2,2,1,34
2,2025-10-08,Tit For Tat,Tate McRae,3,-,3,1
3,2025-10-08,What I Want,Morgan Wallen Featuring Tate McRae,4,4,1,20
4,2025-10-08,Daisies,Justin Bieber,5,8,2,12
...,...,...,...,...,...,...,...
95,2025-10-08,Voy A Llevarte Pa PR,Bad Bunny,96,-,17,12
96,2025-10-08,Nice To Each Other,Olivia Dean,97,-,97,1
97,2025-10-08,So Easy (To Fall In Love),Olivia Dean,98,-,98,1
98,2025-10-08,Take Me Thru Dere,"Metro Boomin, Quavo, Breskii & YKNIECE",99,-,99,1


## 3. Write a loop to retrieve the Billboard chart data for the last 10 weeks.

In [99]:
last_10_df = pd.DataFrame()
thisweek = '2025-11-29' ## for dynamic, calculate the saturday after "today", but I really don't feel like messing with datetimes the day before break

for x in range(0, 11):
    i = x * 7
    returnweek = str((dt.datetime.strptime(thisweek, '%Y-%m-%d') + timedelta(days = -i)).date())
    last_10_df = pd.concat([last_10_df, billboard(returnweek)])

last_10_df = last_10_df.reset_index(drop = True)
last_10_df

,Week,Song,Artist,This Week,Last Week,Peak Position,Weeks on Chart
0,2025-11-29,The Fate Of Ophelia,Taylor Swift,1,1,1,7
1,2025-11-29,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",2,2,1,22
2,2025-11-29,Ordinary,Alex Warren,3,3,1,41
3,2025-11-29,Man I Need,Olivia Dean,4,4,4,13
4,2025-11-29,Opalite,Taylor Swift,5,5,2,7
...,...,...,...,...,...,...,...
1095,2025-09-20,Outside,Cardi B,96,88,10,12
1096,2025-09-20,I Ain't Coming Back,Morgan Wallen Featuring Post Malone,97,87,8,21
1097,2025-09-20,Which One,Drake & Central Cee,98,96,23,7
1098,2025-09-20,Miami,Morgan Wallen Featuring Lil Wayne & Rick Ross,99,85,21,10
